In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/aerial-view-car-detection-for-yolov5/mydata.yaml
/kaggle/input/aerial-view-car-detection-for-yolov5/mydata128 .yaml
/kaggle/input/aerial-view-car-detection-for-yolov5/mydata/mydata/labels/test.cache
/kaggle/input/aerial-view-car-detection-for-yolov5/mydata/mydata/labels/train.cache
/kaggle/input/aerial-view-car-detection-for-yolov5/mydata/mydata/labels/test/3 (48)_1650423600.txt
/kaggle/input/aerial-view-car-detection-for-yolov5/mydata/mydata/labels/test/2 (17)_1649991010.txt
/kaggle/input/aerial-view-car-detection-for-yolov5/mydata/mydata/labels/test/4 (13)_1649859983.txt
/kaggle/input/aerial-view-car-detection-for-yolov5/mydata/mydata/labels/test/1 (4)_1649859559.txt
/kaggle/input/aerial-view-car-detection-for-yolov5/mydata/mydata/labels/test/5 (41)_1650424112.txt
/kaggle/input/aerial-view-car-detection-for-yolov5/mydata/mydata/labels/test/5 (29)_1650425730.txt
/kaggle/input/aerial-view-car-detection-for-yolov5/mydata/mydata/labels/test/3 (45)_1650423933.txt
/kaggle/inp

In [4]:
!wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/{engine.py,utils.py,transforms.py}


--2025-04-05 04:58:10--  https://raw.githubusercontent.com/pytorch/vision/main/references/detection/engine.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4063 (4.0K) [text/plain]
Saving to: ‘engine.py’

engine.py           100%[===================>]   3.97K  --.-KB/s    in 0s      

2025-04-05 04:58:10 (52.3 MB/s) - ‘engine.py’ saved [4063/4063]

--2025-04-05 04:58:10--  https://raw.githubusercontent.com/pytorch/vision/main/references/detection/utils.py
Reusing existing connection to raw.githubusercontent.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 8388 (8.2K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]   8.19K  --.-KB/s    in 0s      

2025-04-05 04:58:11 (47.8 MB/s) - ‘utils.py’ saved [8388/838

In [12]:
import os
import torch
import torchvision
import numpy as np
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torchvision.transforms.functional as F

# Class map
label_map = {"car": 0}
reverse_label_map = {v: k for k, v in label_map.items()}

# Dataset for YOLO-format labels
class YOLODataset(Dataset):
    def __init__(self, img_dir, label_dir, transforms=None):
        self.img_dir = img_dir
        self.label_dir = label_dir
        self.transforms = transforms
        self.imgs = sorted([img for img in os.listdir(img_dir) if img.endswith(".jpg")])

    def __getitem__(self, idx):
        img_name = self.imgs[idx]
        img_path = os.path.join(self.img_dir, img_name)
        label_path = os.path.join(self.label_dir, img_name.replace(".jpg", ".txt"))

        img = Image.open(img_path).convert("RGB")
        w, h = img.size

        boxes = []
        labels = []

        if os.path.exists(label_path):
            with open(label_path, "r") as f:
                for line in f:
                    class_id, xc, yc, bw, bh = map(float, line.strip().split())
                    xc *= w
                    yc *= h
                    bw *= w
                    bh *= h
                    xmin = xc - bw / 2
                    ymin = yc - bh / 2
                    xmax = xc + bw / 2
                    ymax = yc + bh / 2

                    # Skip invalid boxes
                    if xmax > xmin and ymax > ymin:
                        boxes.append([xmin, ymin, xmax, ymax])
                        labels.append(int(class_id) + 1)
                    else:
                        print(f"⚠️ Skipping invalid box: {[xmin, ymin, xmax, ymax]} in {img_name}")

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        iscrowd = torch.zeros((len(labels),), dtype=torch.int64)

        target = {
            "boxes": boxes,
            "labels": labels,
            "image_id": image_id,
            "area": area,
            "iscrowd": iscrowd,
        }

        if self.transforms:
            img = self.transforms(img)

        return img, target

    def __len__(self):
        return len(self.imgs)


def get_transform(train=True):
    def transform(img):
        img = F.to_tensor(img)
        if train and torch.rand(1) < 0.5:
            img = F.hflip(img)
        return img
    return transform


def get_model(num_classes):
    model = fasterrcnn_resnet50_fpn(weights="DEFAULT")
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model


def collate_fn(batch):
    return tuple(zip(*batch))


def train_one_epoch(model, optimizer, data_loader, device, epoch):
    model.train()
    for i, (images, targets) in enumerate(data_loader):
        images = list(img.to(device) for img in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        if i % 10 == 0:
            print(f"Epoch {epoch}, Step {i}, Loss: {losses.item():.4f}")


def evaluate(model, data_loader, device):
    model.eval()
    print("\n[Evaluation]")

    with torch.no_grad():
        for images, targets in data_loader:
            images = list(img.to(device) for img in images)
            outputs = model(images)
            for i, output in enumerate(outputs):
                boxes = output['boxes'].cpu().numpy()
                scores = output['scores'].cpu().numpy()
                print(f"Image ID: {targets[i]['image_id'].item()} | Detections: {len(boxes)}")


def train_model(lr, batch_size, num_epochs=5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    train_dataset = YOLODataset(
        "/kaggle/input/aerial-view-car-detection-for-yolov5/mydata/mydata/images/train",
        "/kaggle/input/aerial-view-car-detection-for-yolov5/mydata/mydata/labels/train",
        get_transform(train=True)
    )

    test_dataset = YOLODataset(
        "/kaggle/input/aerial-view-car-detection-for-yolov5/mydata/mydata/images/test",
        "/kaggle/input/aerial-view-car-detection-for-yolov5/mydata/mydata/labels/test",
        get_transform(train=False)
    )

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, collate_fn=collate_fn)

    num_classes = len(label_map) + 1
    model = get_model(num_classes).to(device)

    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=lr, momentum=0.9, weight_decay=0.0005)
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

    for epoch in range(num_epochs):
        print(f"\n=== Epoch {epoch+1}/{num_epochs} | LR={lr}, Batch Size={batch_size} ===")
        train_one_epoch(model, optimizer, train_loader, device, epoch)
        lr_scheduler.step()

    evaluate(model, test_loader, device)


if __name__ == "__main__":
    for lr in [0.01, 0.005, 0.001]:
        for bs in [2, 4]:
            train_model(lr, bs)



=== Epoch 1/5 | LR=0.01, Batch Size=2 ===
Epoch 0, Step 0, Loss: 3.9410
Epoch 0, Step 10, Loss: 0.7593
Epoch 0, Step 20, Loss: 0.6774
Epoch 0, Step 30, Loss: 1.3931
Epoch 0, Step 40, Loss: 1.2262
Epoch 0, Step 50, Loss: 1.2551
Epoch 0, Step 60, Loss: 0.6266
Epoch 0, Step 70, Loss: 0.6972
Epoch 0, Step 80, Loss: 1.3640
Epoch 0, Step 90, Loss: 0.6930
Epoch 0, Step 100, Loss: 0.8485
Epoch 0, Step 110, Loss: 0.7328
Epoch 0, Step 120, Loss: 0.9910
⚠️ Skipping invalid box: [317.466078, 648.337136, 317.466078, 649.3620520000001] in 2 (49)_1650423572.jpg
Epoch 0, Step 130, Loss: 0.7697
⚠️ Skipping invalid box: [9.92018, 587.758171, 11.56648, 587.758171] in 3 (15)_1649991173.jpg

=== Epoch 2/5 | LR=0.01, Batch Size=2 ===
Epoch 1, Step 0, Loss: 1.1050
⚠️ Skipping invalid box: [9.92018, 587.758171, 11.56648, 587.758171] in 3 (15)_1649991173.jpg
Epoch 1, Step 10, Loss: 0.4292
⚠️ Skipping invalid box: [317.466078, 648.337136, 317.466078, 649.3620520000001] in 2 (49)_1650423572.jpg
Epoch 1, Step 20